In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import pickle
!pip install minisom

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12042 sha256=b45a1f180503e7f597f6babd7f25fb08da367eee6007bb74f7f7cf86c735b371
  Stored in directory: c:\users\r y z e n\appdata\local\pip\cache\wheels\19\db\95\5e53bc2b88a328217fdf9f2886cafbe86b0df274f4b601f572
Successfully built minisom


In [3]:
df = pd.read_csv(r"C:\Users\R Y Z E N\Desktop\indian_liver_patient.csv")


In [5]:
 #Fill missing values
df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].mean(), inplace=True)


In [6]:
# Encode Gender
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])

In [7]:
# Convert 'Dataset' to binary (1 = disease, 0 = no disease)
df['Dataset'] = df['Dataset'].apply(lambda x: 1 if x == 1 else 0)


In [8]:
# Convert 'Dataset' to binary (1 = disease, 0 = no disease)
df['Dataset'] = df['Dataset'].apply(lambda x: 1 if x == 1 else 0)


In [10]:
# Split features and target
X = df.drop("Dataset", axis=1)
y = df["Dataset"]


In [11]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [12]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [13]:
perceptron = Perceptron(max_iter=1000, eta0=0.1, random_state=0)
perceptron.fit(X_train, y_train)
print("Perceptron Results:\n", classification_report(y_test, perceptron.predict(X_test)))


Perceptron Results:
               precision    recall  f1-score   support

           0       0.37      0.93      0.53        30
           1       0.95      0.45      0.61        87

    accuracy                           0.57       117
   macro avg       0.66      0.69      0.57       117
weighted avg       0.80      0.57      0.59       117



In [14]:

mlp = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam',
                    alpha=0.001, max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
print("MLP Results:\n", classification_report(y_test, mlp.predict(X_test)))


MLP Results:
               precision    recall  f1-score   support

           0       0.46      0.60      0.52        30
           1       0.85      0.76      0.80        87

    accuracy                           0.72       117
   macro avg       0.65      0.68      0.66       117
weighted avg       0.75      0.72      0.73       117



C:\Users\R Y Z E N\.matplotlib\ana1\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
from minisom import MiniSom

som = MiniSom(x=10, y=10, input_len=X_train.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_train)
som.train_random(X_train, 100)

In [16]:

class RBFNetwork(BaseEstimator, ClassifierMixin):
    def __init__(self, gamma=1.0):
        self.gamma = gamma
    
    def _rbf(self, X, centers):
        return np.exp(-self.gamma * np.linalg.norm(X[:, np.newaxis] - centers, axis=2) ** 2)

    def fit(self, X, y):
        self.centers = X[np.random.choice(X.shape[0], 10, replace=False)]
        self.X_trans = self._rbf(X, self.centers)
        self.model = SVC(kernel='linear')
        self.model.fit(self.X_trans, y)
        return self

    def predict(self, X):
        X_rbf = self._rbf(X, self.centers)
        return self.model.predict(X_rbf)

rbf_net = RBFNetwork(gamma=0.1)
rbf_net.fit(X_train, y_train)
print("RBFN Results:\n", classification_report(y_test, rbf_net.predict(X_test)))


RBFN Results:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        30
           1       0.74      1.00      0.85        87

    accuracy                           0.74       117
   macro avg       0.37      0.50      0.43       117
weighted avg       0.55      0.74      0.63       117



C:\Users\R Y Z E N\.matplotlib\ana1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\R Y Z E N\.matplotlib\ana1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\R Y Z E N\.matplotlib\ana1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:

class BAM:
    def __init__(self):
        self.weights = None

    def train(self, x, y):
        self.weights = np.dot(x.T, y)

    def recall(self, x):
        y = np.sign(np.dot(x, self.weights))
        return y

# Convert input to bipolar (-1, 1)
X_bam = np.where(X > 0, 1, -1)
y_bam = np.where(y.values.reshape(-1, 1) > 0, 1, -1)

bam = BAM()
bam.train(X_bam, y_bam)
y_pred_bam = bam.recall(X_bam)


In [18]:
models = {
    "perceptron": perceptron,
    "mlp": mlp,
    "rbfn": rbf_net,
    "som": som,
    "bam": bam,
    "scaler": scaler
}

with open("ai_health_advisor_models.pkl", "wb") as f:
    pickle.dump(models, f)